In [ ]:

#folder1 = "C:/Users/HP14/Desktop/Electronics/damn/images"
#folder2 = "C:/Users/HP14/Desktop/Electronics/damn/labels"

import os

def find_complement_intersection_names(folder1, folder2):
  """
  This function finds the complement of the intersection of filenames (excluding extensions) in two folders.

  Args:
      folder1 (str): Path to the first folder.
      folder2 (str): Path to the second folder.

  Returns:
      set: Set of filenames (without extensions) that are in one folder but not the other (complement of intersection).
  """
  all_filenames = set(os.path.splitext(f)[0] for f in os.listdir(folder1)) | set(os.path.splitext(f)[0] for f in os.listdir(folder2))
  intersection = all_filenames.intersection(set(os.path.splitext(f)[0] for f in os.listdir(folder1))) & all_filenames.intersection(set(os.path.splitext(f)[0] for f in os.listdir(folder2)))
  complement = all_filenames - intersection

  return complement

# Example usage
folder1 = "C:/Users/HP14/Desktop/Electronics/damn/images"
folder2 = "C:/Users/HP14/Desktop/Electronics/damn/labels"

complement_names = find_complement_intersection_names(folder1, folder2)

if complement_names:
  print("Complement of intersection (filenames without extensions in one folder but not the other):")
  for filename in complement_names:
    print(filename)
else:
  print("No filenames found in one folder but not the other")

print("Finished checking files.")


 

In [ ]:
import os
os.chdir("C:/Users/HP14/Desktop/Electronics")


## old but working (without api)

In [ ]:
from ultralytics import YOLO
import cv2
import math
import time
import os
from datetime import datetime

# start webcam
cap = cv2.VideoCapture(0)
cap.set(3, 640)
cap.set(4, 480)

# model
model = YOLO("BestModel.pt")

# object classes
classNames = ["gun", "knife"]

# confidence threshold
confidence_threshold = 0.5

# specify the directory to save screenshots
screenshot_directory = "screenshots/"

# create the directory if it doesn't exist
if not os.path.exists(screenshot_directory):
    os.makedirs(screenshot_directory)

while True:
    success, img = cap.read()
    if not success:
        break

    results = model(img, stream=True)

    # coordinates
    for r in results:
        boxes = r.boxes

        for box in boxes:
            # bounding box
            x1, y1, x2, y2 = box.xyxy[0]
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)  # convert to int values

            # put box in cam
            cv2.rectangle(img, (x1, y1), (x2, y2), (0, 0, 255), 3)

            # confidence
            confidence = math.ceil((box.conf[0] * 100)) / 100
            print("Confidence --->", confidence)

            # class name
            cls = int(box.cls[0])
            print("Class name -->", classNames[cls])

            # if confidence is above the threshold, save a screenshot
            if confidence > confidence_threshold:
                current_time = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
                screenshot_filename = f"screenshot_{classNames[cls]}_{current_time}.jpg"
                full_path = os.path.join(screenshot_directory, screenshot_filename)
                cv2.imwrite(full_path, img)
                print(f"Screenshot saved as {full_path}")

            # prepare text for display (class name + confidence)
            label = f"{classNames[cls]}: {confidence:.2f}"

            # object details for text placement
            org = [x1, y1 - 10]  # position text above the bounding box
            font = cv2.FONT_HERSHEY_SIMPLEX
            fontScale = 0.9
            color = (255, 255, 255)  # white text
            thickness = 2

            # put the label (class name + confidence) on the image
            cv2.putText(img, label, org, font, fontScale, color, thickness)

    # display webcam feed
    cv2.imshow('Webcam', img)
    if cv2.waitKey(1) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [ ]:
from ultralytics import YOLO
import cv2
import math
import os
from datetime import datetime
import requests
import json

# Start webcam
cap = cv2.VideoCapture(0)
cap.set(3, 640)
cap.set(4, 480)

# Load YOLO model
model = YOLO("BestModel.pt")

# Object classes
classNames = ["gun", "knife"]

# Confidence threshold
confidence_threshold = 0.5

# Specify the directory to save screenshots
screenshot_directory = "screenshots/"

# Create the directory if it doesn't exist
if not os.path.exists(screenshot_directory):
    os.makedirs(screenshot_directory)

# API endpoint
api_url = "http://localhost:7125/api/DetectAPI"

def get_binary_object_type(class_name):
    if class_name == "gun":
        return 0 
    else:
        return 1

while True:
    success, img = cap.read()
    if not success:
        break

    results = model(img, stream=True)

    for r in results:
        boxes = r.boxes

        for box in boxes:
            # Bounding box
            x1, y1, x2, y2 = box.xyxy[0]
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)

            # Draw rectangle
            cv2.rectangle(img, (x1, y1), (x2, y2), (0, 0, 255), 3)

            # Confidence
            confidence = math.ceil((box.conf[0] * 100)) / 100
            print("Confidence --->", confidence)

            # Class name and binary representation
            cls = int(box.cls[0])
            class_name = classNames[cls]
            binary_type = get_binary_object_type(class_name)
            print(f"Class name --> {class_name}")

            if confidence > confidence_threshold:
                current_time = datetime.now().isoformat()
                screenshot_filename = f"screenshot_{class_name}_{current_time.replace(':', '-')}.jpg"
                full_path = os.path.join(screenshot_directory, screenshot_filename)
                cv2.imwrite(full_path, img)
                print(f"Screenshot saved as {full_path}")

                # Prepare data to send to API
                data = {
                    "WeaponType": binary_type,  # 0 for gun, 1 for knife
                    "Confidence": confidence,
                    "Timestamp": current_time
                }

                # Send data to API
                try:
                    headers = {'Content-Type': 'application/json'}
                    response = requests.post(api_url, json=data, headers=headers)
                    if response.status_code == 201:  # 201 is the status code for "Created"
                        print("Data sent to API successfully")
                    else:
                        print(f"Failed to send data to API. Status code: {response.status_code}")
                        print(f"Response content: {response.text}")
                except requests.exceptions.RequestException as e:
                    print(f"Error sending data to API: {e}")

            # Prepare label
            label = f"{class_name} ({binary_type}): {confidence:.2f}"

            # Put label on image
            cv2.putText(img, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 255, 255), 2)

    # Display webcam feed
    cv2.imshow('Webcam', img)
    if cv2.waitKey(1) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

## Working with APIKEY

In [ ]:
from ultralytics import YOLO
import cv2
import math
import os
from datetime import datetime
import requests
import json
import logging
import warnings

# Disable warnings about insecure requests
warnings.filterwarnings('ignore', message='Unverified HTTPS request')

# Configure logging
logging.basicConfig(filename='detection_log.txt', level=logging.INFO, 
                    format='%(asctime)s - %(levelname)s - %(message)s')

# API endpoint
api_url = "https://localhost:7125/api/DetectAPI"

# API Key - replace with your actual API key
API_KEY = "KIahzlPmmgK0Ue4U8H4Z7wfCRcVQargn"

# Start webcam
cap = cv2.VideoCapture(0)
cap.set(3, 640)
cap.set(4, 480)

# Load YOLO model
model = YOLO("BestModel.pt")

# Object classes
classNames = ["gun", "knife"]

# Confidence threshold
confidence_threshold = 0.5

# Specify the directory to save screenshots
screenshot_directory = "screenshots/"

# Create the directory if it doesn't exist
if not os.path.exists(screenshot_directory):
    os.makedirs(screenshot_directory)

def get_binary_object_type(class_name):
    return False if class_name == "gun" else True

logging.info("Script started")

while True:
    success, img = cap.read()
    if not success:
        logging.error("Failed to capture image from camera")
        break

    results = model(img, stream=True)

    for r in results:
        boxes = r.boxes

        for box in boxes:
            # Confidence
            confidence = math.ceil((box.conf[0] * 100)) / 100
            
            # Class name and binary representation
            cls = int(box.cls[0])
            class_name = classNames[cls]
            binary_type = get_binary_object_type(class_name)
            
            logging.info(f"Detection: {class_name}, Confidence: {confidence}")

            if confidence > confidence_threshold:
                current_time = datetime.now().isoformat()
                screenshot_filename = f"screenshot_{class_name}_{current_time.replace(':', '-')}.jpg"
                full_path = os.path.join(screenshot_directory, screenshot_filename)
                cv2.imwrite(full_path, img)
                logging.info(f"Screenshot saved as {full_path}")

                # Prepare data to send to API
                data = {
                    
                    "WeaponType": binary_type,  
                    "Confidence": float(confidence),
                    "Timestamp": current_time
                                
                }

                # Log the data before sending
                logging.info(f"Sending data to API: {json.dumps(data)}")

                # Send data to API with API Key authentication
                try:
                    headers = {
                        'Content-Type': 'application/json',
                        'X-API-Key': API_KEY
                    }
                    response = requests.post(api_url, json=data, headers=headers, verify=False)
                    logging.info(f"API Response Code: {response.status_code}")
                    logging.info(f"API Response Body: {response.text}")

                    if response.status_code == 201:  # 201 is the status code for "Created"
                        logging.info("Data sent to API successfully")
                    else:
                        logging.error(f"Failed to send data to API. Status code: {response.status_code}")
                        logging.error(f"Response content: {response.text}")
                except requests.exceptions.RequestException as e:
                    logging.error(f"Exception occurred: {e}")
                    logging.error(f"Headers: {headers}")
                    logging.error(f"Payload: {data}")

            # Bounding box
            x1, y1, x2, y2 = box.xyxy[0]
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)

            # Draw rectangle
            cv2.rectangle(img, (x1, y1), (x2, y2), (0, 0, 255), 3)

            # Prepare label
            label = f"{class_name} : {confidence:.2f}"

            # Put label on image
            cv2.putText(img, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 255, 255), 2)

    # Display webcam feed
    cv2.imshow('Webcam', img)
    if cv2.waitKey(1) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

## 10 seconds (Best one)

In [1]:
from ultralytics import YOLO
import cv2
import math
import os
from datetime import datetime
import requests
import json
import logging
import warnings
import time

# Disable warnings about insecure requests
warnings.filterwarnings('ignore', message='Unverified HTTPS request')

# Configure logging
logging.basicConfig(filename='detection_log.txt', level=logging.INFO, 
                    format='%(asctime)s - %(levelname)s - %(message)s')

# API endpoint
api_url = "https://localhost:7125/api/DetectAPI"

# API Key - replace with your actual API key
API_KEY = "KIahzlPmmgK0Ue4U8H4Z7wfCRcVQargn"

# Start webcam
cap = cv2.VideoCapture(0)
cap.set(3, 640)
cap.set(4, 480)

# Load YOLO model
model = YOLO("BestModel.pt")

# Object classes
classNames = ["gun", "knife"]

# Confidence threshold
confidence_threshold = 0.5

# Specify the directory to save screenshots
screenshot_directory = "screenshots/"

# Create the directory if it doesn't exist
if not os.path.exists(screenshot_directory):
    os.makedirs(screenshot_directory)

def get_binary_object_type(class_name):
    return False if class_name == "gun" else True

logging.info("Script started")

# Initialize variables for best detection tracking
last_api_call_time = time.time()
best_detection = None
best_confidence = 0
best_image = None

while True:
    success, img = cap.read()
    if not success:
        logging.error("Failed to capture image from camera")
        break

    results = model(img, stream=True)

    current_time = time.time()
    
    for r in results:
        boxes = r.boxes

        for box in boxes:
            # Confidence
            confidence = math.ceil((box.conf[0] * 100)) / 100
            
            # Class name
            cls = int(box.cls[0])
            class_name = classNames[cls]
            
            logging.info(f"Detection: {class_name}, Confidence: {confidence}")

            if confidence > confidence_threshold:
                # Update best detection if this one has higher confidence
                if confidence > best_confidence:
                    best_detection = {
                        "class_name": class_name,
                        "confidence": confidence,
                        "box": box.xyxy[0].tolist()
                    }
                    best_confidence = confidence
                    best_image = img.copy()

                # Bounding box
                x1, y1, x2, y2 = box.xyxy[0]
                x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)

                # Draw rectangle
                cv2.rectangle(img, (x1, y1), (x2, y2), (0, 0, 255), 3)

                # Prepare label
                label = f"{class_name} : {confidence:.2f}"

                # Put label on image
                cv2.putText(img, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 255, 255), 2)

    # Check if 6 seconds have passed
    if current_time - last_api_call_time >= 10:
        if best_detection:
            current_datetime = datetime.now().isoformat()
            screenshot_filename = f"screenshot_{best_detection['class_name']}_{current_datetime.replace(':', '-')}.jpg"
            full_path = os.path.join(screenshot_directory, screenshot_filename)
            cv2.imwrite(full_path, best_image)
            logging.info(f"Screenshot saved as {full_path}")

            # Prepare data to send to API
            data = {
                "WeaponType": get_binary_object_type(best_detection['class_name']),  
                "Confidence": float(best_detection['confidence']),
                "Timestamp": current_datetime,
                "ImagePath": f"C:/Users/HP14/Desktop/GP/screenshots/{screenshot_filename}"
            }

            # Log the data before sending
            logging.info(f"Sending best detection to API: {json.dumps(data)}")

            # Send data to API with API Key authentication
            try:
                headers = {
                    'Content-Type': 'application/json',
                    'X-API-Key': API_KEY
                }
                response = requests.post(api_url, json=data, headers=headers, verify=False)
                logging.info(f"API Response Code: {response.status_code}")
                logging.info(f"API Response Body: {response.text}")

                if response.status_code == 201:  # 201 is the status code for "Created"
                    logging.info("Data sent to API successfully")
                else:
                    logging.error(f"Failed to send data to API. Status code: {response.status_code}")
                    logging.error(f"Response content: {response.text}")
            except requests.exceptions.RequestException as e:
                logging.error(f"Exception occurred: {e}")
                logging.error(f"Headers: {headers}")
                logging.error(f"Payload: {data}")

        # Reset best detection tracking
        last_api_call_time = current_time
        best_detection = None
        best_confidence = 0
        best_image = None

    # Display webcam feed
    cv2.imshow('Webcam', img)
    if cv2.waitKey(1) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


0: 480x640 (no detections), 850.9ms
Speed: 25.7ms preprocess, 850.9ms inference, 14.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 575.0ms
Speed: 6.0ms preprocess, 575.0ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 493.9ms
Speed: 4.9ms preprocess, 493.9ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 496.6ms
Speed: 6.5ms preprocess, 496.6ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 519.2ms
Speed: 4.1ms preprocess, 519.2ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 524.6ms
Speed: 6.2ms preprocess, 524.6ms inference, 2.6ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 610.9ms
Speed: 7.2ms preprocess, 610.9ms inference, 1.6ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 585.1ms
Speed: 8.1ms pre